# IMDB Web Scrape

In [1]:
import requests
from bs4 import BeautifulSoup as BS

In [2]:
url = 'https://www.imdb.com/search/title/?title_type=feature,tv_movie&genres=horror&view=advanced'
headers = {'user-agent': 'Safari/13.0.2 (Macintosh; Intel Mac OS X 10_15)'}
page = requests.get(url,headers=headers, timeout = 5)
soup = BS(page.content, 'html.parser')

In [24]:
boxes_50 = soup.find(class_='lister list detail sub-list').find_all(class_='lister-item mode-advanced')

In [63]:
boxes_50[0].find('a').get_text

<bound method Tag.get_text of <a href="/title/tt1560220/?ref_=adv_li_i"> <img alt="Zombieland: Double Tap" class="loadlate" data-tconst="tt1560220" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYjcwNjZhNGYtOGNlNy00NGI3LTlmODMtMGZlMjA3YjA0Njg0XkEyXkFqcGdeQXVyMDM2NDM2MQ@@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a>>

In [80]:
boxes_50[0].find(class_=['span', 'metascore mixed']).text.strip()

'56'

In [ ]:
for movie in movies:
    details = {}
    details['url'] = 'www.imdb.com/' + movie.find('a')['href'] # Assigns url to website
    details['metacritic'] = movie.find(class_=['span', 'metascore mixed']).text.strip() # Assigns metacritic score